<a href="https://colab.research.google.com/github/Athugodage/RuLawSimplification/blob/main/GPT/evaluate_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 KB 6.5 MB/s eta 0:00:00


In [1]:
!pip install transformers
!pip install rouge_score sacrebleu sacremoses
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 44.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=29963f78582a2dba06f0f5ff9c52b91f0c7b5c2ea02237fe2fa610686dd10611
  Stored in directory: /root/.cache/pip/wheels/9b/3d/39/09558097d3119ca0a4d462df68f22c6f3c1b345ac63a09b86e
  Created wheel for sacremoses: filename=sacremoses-

In [2]:
!git clone https://github.com/Athugodage/RuLawSimplification.git

Cloning into 'RuLawSimplification'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (269/269), done.
remote: Compressing objects: 100% (183/183), done.
remote: Total 535 (delta 138), reused 152 (delta 79), pack-reused 266
Receiving objects: 100% (535/535), 243.92 MiB | 17.97 MiB/s, done.
Resolving deltas: 100% (251/251), done.


In [3]:
!unzip '/content/RuLawSimplification/corpus/cut_corpus.zip'

Archive:  /content/RuLawSimplification/corpus/cut_corpus.zip
  inflating: cut_corpus.csv          


In [44]:
import pandas as pd

df = pd.read_csv('cut_corpus.csv')

In [45]:
df = df.dropna()

In [46]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, random_state=42, test_size=0.05)

In [47]:
train.columns = ["text", "commentary"]
test.columns = ["text", "commentary"]

In [8]:
from transformers import pipeline

In [43]:
text_generator = pipeline("text-generation", 
                          model='marcus2000/GPT_simplifier25', 
                          tokenizer='sberbank-ai/rugpt3small_based_on_gpt2')

# Generate text

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
prompt = test.iloc[0]["text"][:2400] + '[SEP] '

In [ ]:
generated_text = text_generator(prompt, max_length=1000, repetition_penalty=1.0, do_sample=False)
print(generated_text)

In [ ]:
generated_text

[{'generated_text': ' При этом, обязывая государственные власти воздерживаться от всякого вмешательства, способного ограничить право на объединение или воспрепятствовать его законному осуществлению, данная Конвенция рассматривает в качестве элемента его содержания право организаций трудящихся и предпринимателей самостоятельно вырабатывать свои уставы и административные регламенты, свободно выбирать своих представителей, организовывать свой аппарат и свою деятельность, формулировать свою программу действий и гарантирует указанные права также федерациям и конфедерациям организаций трудящихся и предпринимателей (статьи 3 и 6). Исходя из этого и в силу статей 1 (часть 1), 2, 15 (часть 4), 17 (часть 1), 18, 30 и 55 (часть 3) Конституции Российской Федерации федеральный законодатель при осуществлении правового регулирования отношений, связанных с реализацией гражданами конституционного права на объединение в профессиональные союзы, созданием и деятельностью профессиональных союзов и их объед

In [ ]:
len(test["commentary"])

93

In [48]:
from tqdm import tqdm

real = test['text'].to_list()
preds = []

for orig in tqdm(real):
    generated_text = text_generator(orig[:2500], max_length=1000, repetition_penalty=1.0)
    generated_text = str(generated_text[0]['generated_text']).split("[SEP] ")[1:]
    preds.append(generated_text)



 18%|█▊        | 17/93 [11:28<47:02, 37.14s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1213, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
100%|██████████| 93/93 [1:00:17<00:00, 38.90s/it]


In [ ]:
import evaluate

sari = evaluate.load("sari")

real = [list(orig[:2500]) for orig in test['text']]


sari.compute(sources=test['text'].to_list(),  ## original
             references=[[ref] for ref in test['commentary'].to_list()], ## the comment from RG
             predictions=[str(p) for p in preds]  ## model's version
             )

{'sari': 40.965917447138395}

In [ ]:
rouge = evaluate.load("rouge")
rouge.compute(references=test['text'].to_list(), predictions=[str(p) for p in preds])

{'rouge1': 0.050720799371253684,
 'rouge2': 0.009823308466685079,
 'rougeL': 0.04693134201289889,
 'rougeLsum': 0.046370965136756634}

In [52]:
from evaluate import load
bertscore = load("bertscore")

results = bertscore.compute(predictions=[str(p) for p in preds], 
                            references=[[ref] for ref in test['commentary'].to_list()], lang='ru')

In [17]:
results

{'precision': [0.7408359050750732,
  0.542073130607605,
  0.6467236876487732,
  0.6242067813873291,
  0.7165180444717407,
  0.7257958054542542,
  0.6309124827384949,
  0.6342910528182983,
  0.6654045581817627,
  0.6142334938049316,
  0.7033154368400574,
  0.6084568500518799,
  0.6963027715682983,
  0.6510747671127319,
  0.6695857644081116,
  0.5865983963012695,
  0.5379906892776489,
  0.48743653297424316,
  0.6873959898948669,
  0.6263144612312317,
  0.6618980169296265,
  0.6198432445526123,
  0.646277904510498,
  0.7176063060760498,
  0.6451316475868225,
  0.7053526639938354,
  0.685310959815979,
  0.6626693606376648,
  0.8201364278793335,
  0.6890645027160645,
  0.643937349319458,
  0.7106426954269409,
  0.648639440536499,
  0.6535667777061462,
  0.7534561157226562,
  0.6496905088424683,
  0.7122853398323059,
  0.6882227659225464,
  0.6270362138748169,
  0.6250671148300171,
  0.6116323471069336,
  0.7008638978004456,
  0.6468048095703125,
  0.5837111473083496,
  0.6290551424026489,
 

In [53]:
from statistics import mean
mean(results['precision'])

0.6191451360461533

In [54]:
from statistics import mean
mean(results['f1'])

0.6041428706338329

In [55]:
## GUNNING FOG READABILITY INDEX

def gunning_fog(text):
  text = str(text)

  tokens = nltk.word_tokenize(text)
  tokens_length = len(tokens)
  sent_length = len(nltk.sent_tokenize(text))
  complex_tokens_len = len([tok for tok in tokens if len(tok) > 6])
  formule = 0.4 * (tokens_length / sent_length) + 100 * (complex_tokens_len / tokens_length)
  return formule


In [56]:
indexes = []

## original

for t in test['text'].to_list():
  indexes.append(gunning_fog(t))

print(mean(indexes))

59.56413033697357


In [57]:
indexes = []

## simplified by model

for pred in preds:
  indexes.append(gunning_fog(pred))

print(mean(indexes))

41.83912856502404


uncut


In [18]:
!unzip '/content/RuLawSimplification/corpus/the_corpus.zip'

Archive:  /content/RuLawSimplification/corpus/the_corpus.zip
  inflating: the_corpus.csv          


In [19]:
df = pd.read_csv('the_corpus.csv')

In [20]:
text_generator = pipeline("text-generation", 
                          model='marcus2000/GPT_simplifier_large_text', 
                          tokenizer='sberbank-ai/rugpt3small_based_on_gpt2')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [21]:
from tqdm import tqdm

real = test['text'].to_list()
preds = []

for orig in tqdm(real):
    generated_text = text_generator(orig[:2500] + '[SEP] ', max_length=1000)
    generated_text = str(generated_text[0]['generated_text']).split("[SEP] ")[1:]
    preds.append(generated_text)

 18%|█▊        | 17/93 [10:04<44:02, 34.77s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Input length of input_ids is 1219, but `max_length` is set to 1000. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
100%|██████████| 93/93 [57:02<00:00, 36.81s/it]


In [ ]:
import evaluate

sari = evaluate.load("sari")

real = [list(orig[:2500]) for orig in test['text']]


sari.compute(sources=test['text'].to_list(),  ## original
             references=[[ref] for ref in test['commentary'].to_list()], ## the comment from RG
             predictions=[str(p) for p in preds]  ## model's version
             )

{'sari': 39.907188213348654}

In [ ]:
rouge = evaluate.load("rouge")
rouge.compute(references=test['text'].to_list(), predictions=[str(p) for p in preds])

{'rouge1': 0.07365608088048531,
 'rouge2': 0.014426468385822993,
 'rougeL': 0.0690339414274968,
 'rougeLsum': 0.0689156230010729}

In [ ]:
from tqdm import tqdm

real = test['text'].to_list()
preds = []

for orig in tqdm(real):
    generated_text = text_generator(orig[:2500] + '[SEP] ', max_length=1000)
    generated_text = str(generated_text[0]['generated_text']).split("[SEP] ")[1:]
    preds.append(generated_text)

In [ ]:
import evaluate

sari = evaluate.load("sari")

real = [list(orig[:2500]) for orig in test['text']]


sari.compute(sources=test['text'].to_list(),  ## original
             references=[[ref] for ref in test['commentary'].to_list()], ## the comment from RG
             predictions=[str(p) for p in preds]  ## model's version
             )

In [ ]:
rouge = evaluate.load("rouge")
rouge.compute(references=test['text'].to_list(), predictions=[str(p) for p in preds])

In [32]:
from evaluate import load
bertscore = load("bertscore")

results = bertscore.compute(predictions=[str(p) for p in preds], 
                            references=[[ref] for ref in test['commentary'].to_list()], lang='ru')

In [33]:
from statistics import mean
mean(results['precision'])

0.6155967558583906

In [34]:
from statistics import mean
mean(results['f1'])

0.6025642294396636

Индекс наводящего тумана

далее используют следующую формулу:


0.4* (кол-во токенов/кол-во предложений)+100(кол-во сложных токенов/кол-во токенов)

In [39]:
import nltk
nltk.download('punkt')

def gunning_fog(text):
  text = str(text)

  tokens = nltk.word_tokenize(text)
  tokens_length = len(tokens)
  sent_length = len(nltk.sent_tokenize(text))
  complex_tokens_len = len([tok for tok in tokens if len(tok) > 6])
  formule = 0.4 * (tokens_length / sent_length) + 100 * (complex_tokens_len / tokens_length)
  return formule


gunning_fog(preds[0])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


33.00055517002082

In [40]:
indexes = []

## simplified by model

for pred in preds:
  indexes.append(gunning_fog(pred))

print(mean(indexes))

54.08566165689718


In [42]:
indexes = []

## original text

for t in test['text'].to_list():
  indexes.append(gunning_fog(t))

print(mean(indexes))

59.56413033697357
